In [1]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import glob
import pickle

%matplotlib inline

In [2]:
class FrameData():
    def __init__(self, frame):
        self.img_size = (frame.shape[1], frame.shape[0])
        self.center_x = self.img_size[0]/2
        self.center_y = self.img_size[1]/2

In [3]:
def adjust_camera():
    # prepare object points
    objp = np.zeros((6*9,3), np.float32)
    objp[:,:2] = np.mgrid[0:9, 0:6].T.reshape(-1,2)
    
    # arrays to store object points and image points from all the images.
    objpoints = [] # 3d points in real world space
    imgpoints = [] # 3d points in image plane.

    # make a list of calibration image
    images = glob.glob('camera_cal/calibration*.jpg')

    # step through the list and search for chessboard corners
    for idx, fname in enumerate(images):
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

        # find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, (9,6), None)

        # if found, add object points, image points
        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)
        
    return objpoints, imgpoints

In [4]:
def calibrate(objpoints, imgpoints, img_size):    
    # do camera calibration given object points and image points
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size, None, None)
    return mtx, dist


In [19]:
# Edit this function to create your own pipeline.
def screening(img, s_thresh=(90, 255), sx_thresh=(20, 100)):
    img = np.copy(img)
    # Convert to HSV color space and separate the V channel
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HLS).astype(np.float)
    l_channel = hsv[:,:,1]
    s_channel = hsv[:,:,2]
    # Sobel x
    sobelx = cv2.Sobel(l_channel, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1
    
    # Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    # Stack each channel
    # Note color_binary[:, :, 0] is all 0s, effectively an all black image. It might
    # be beneficial to replace this channel with something else.
    color_binary = np.dstack(( np.zeros_like(sxbinary), sxbinary, s_binary))
    
    # Combine the two binary thresholds
    combined_binary = np.zeros_like(sxbinary)
    combined_binary[(s_binary == 1) | (sxbinary == 1)] = 1

    return color_binary, combined_binary
#     return color_binary, s_binary

In [20]:
class Perspective:
    def __init__(self, img_size, center_x, center_y):
        self.src = np.float32(
            [[center_x+60,center_y+120],
             [img_size[0]*(6/7),img_size[1]],
             [img_size[0]*(1/5),img_size[1]],
             [center_x,center_y+120]])
        self.dst = np.float32([
            [img_size[0]*(3/4),0], 
            [img_size[0]*(3/4),img_size[1]],   
            [img_size[0]*(1/4),img_size[1]], 
            [img_size[0]*(1/4),0]])

In [21]:
def draw_line(img, src, color=[255,0,0]):
    x1, y1 = src[0]
    x2, y2 = src[1]
    cv2.line(img, (x1, y1), (x2, y2), color, 5)
    x3, y3 = src[2]
    x4, y4 = src[3]
    cv2.line(img, (x3, y3), (x4, y4), color, 5)
    return img

def transform(binary, img_size, prsp):
    M = cv2.getPerspectiveTransform(prsp.src, prsp.dst)
    binary_warped = cv2.warpPerspective(binary, M, img_size, flags=cv2.INTER_LINEAR)
    return binary_warped

In [22]:
def find_curves(binary_warped,lines,midpoint):
    if lines[0].detected and lines[1].detected: # after taking the first frame of the video
        # Assume you now have a new warped binary image 
        # from the next frame of video (also called "binary_warped")
        # It's now much easier to find line pixels!
        nonzero = binary_warped.nonzero()
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        margin = 70
        left_fit = lines[0].recent_fitted[-1]
        right_fit = lines[1].recent_fitted[-1]
        left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + 
                                       left_fit[1]*nonzeroy + left_fit[2] - margin)) &
                          (nonzerox < (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] + margin))) 
        right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + 
                                        right_fit[1]*nonzeroy + right_fit[2] - margin)) &
                           (nonzerox < (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] + margin)))  

        # Again, extract left and right line pixel positions
        leftx = nonzerox[left_lane_inds]
        lefty = nonzeroy[left_lane_inds] 
        rightx = nonzerox[right_lane_inds]
        righty = nonzeroy[right_lane_inds]
        # if leftx[-1] and lefty[-1] crossed, it's illigal line detection
        
        # Fit a second order polynomial to each
        left_fit = np.polyfit(lefty, leftx, 2)
        right_fit = np.polyfit(righty, rightx, 2)
    else:
        # Assuming you have created a warped binary image called "binary_warped"
        # Take a histogram of the bottom half of the image
        histogram = np.sum(binary_warped[binary_warped.shape[0]/2:,:], axis=0)
        # Create an output image to draw on and  visualize the result
        out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
        # Find the peak of the left and right halves of the histogram
        # These will be the starting point for the left and right lines
        midpoint = np.int(histogram.shape[0]/2)
        leftx_base = np.argmax(histogram[:midpoint])
        rightx_base = np.argmax(histogram[midpoint:]) + midpoint

        # Choose the number of sliding windows
        nwindows = 27
        # Set height of windows
        window_height = np.int(binary_warped.shape[0]/nwindows)
        # Identify the x and y positions of all nonzero pixels in the image
        nonzero = binary_warped.nonzero()
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        # Current positions to be updated for each window
        leftx_current = leftx_base
        rightx_current = rightx_base
        # Set the width of the windows +/- margin
        margin = 70
        # Set minimum number of pixels found to recenter window
        minpix = 150
        # Create empty lists to receive left and right lane pixel indices
        left_lane_inds = []
        right_lane_inds = []

        # Step through the windows one by one
        for window in range(nwindows):
            # Identify window boundaries in x and y (and right and left)
            win_y_low = binary_warped.shape[0] - (window+1)*window_height
            win_y_high = binary_warped.shape[0] - window*window_height
            win_xleft_low = leftx_current - margin
            win_xleft_high = leftx_current + margin
            win_xright_low = rightx_current - margin
            win_xright_high = rightx_current + margin
            # Draw the windows on the visualization image
            cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(0,255,0), 2) 
            cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),(0,255,0), 2) 
            # Identify the nonzero pixels in x and y within the window
            good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) &
                              (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
            good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) &
                               (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
            # Append these indices to the lists
            left_lane_inds.append(good_left_inds)
            right_lane_inds.append(good_right_inds)
            # If you found > minpix pixels, recenter next window on their mean position
            if len(good_left_inds) > minpix:
                leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
            if len(good_right_inds) > minpix:        
                rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

        # Concatenate the arrays of indices
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)

        # Extract left and right line pixel positions
        leftx = nonzerox[left_lane_inds]
        lefty = nonzeroy[left_lane_inds] 
        rightx = nonzerox[right_lane_inds]
        righty = nonzeroy[right_lane_inds] 

        # Fit a second order polynomial to each
        left_fit = np.polyfit(lefty, leftx, 2)
        right_fit = np.polyfit(righty, rightx, 2)

    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

    leftx = left_fitx[::-1]  # Reverse to match top-to-bottom in y
    rightx = right_fitx[::-1]  # Reverse to match top-to-bottom in y

    # Define y-value where we want radius of curvature
    # I'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = np.max(ploty)
    # left_curverad = ((1 + (2*left_fit[0]*y_eval + left_fit[1])**2)**1.5) / np.absolute(2*left_fit[0])
    # right_curverad = ((1 + (2*right_fit[0]*y_eval + right_fit[1])**2)**1.5) / np.absolute(2*right_fit[0])
    # print(left_curverad, right_curverad)

    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension

    # Fit new polynomials to x,y in world space
    left_fit_cr = np.polyfit(ploty*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(ploty*ym_per_pix, rightx*xm_per_pix, 2)
    # Calculate the new radii of curvature
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])

    lines[0].detected = True
    lines[0].recent_xfitted.append(left_fitx)
    lines[0].recent_fitted.append(left_fit)
    lines[0].best_fit = np.mean(lines[0].recent_fitted)
    lines[0].current_fit = left_fit
    lines[0].radius_of_curvature = left_curverad
#     lines[0].line_base_pos = (midpoint -leftx[0])*xm_per_pix
    lines[0].line_base_pos = leftx[0]
    lines[0].allx = leftx
    lines[0].ally = lefty
    if lines[0].detected:
        lines[0].diffs = left_fit - lines[0].recent_fitted[-1]
    if len(lines[0].recent_xfitted) > 3:
        lines[0].bestx = np.mean(lines[0].recent_xfitted[-3:], axis=0)
    
    lines[1].detected = True
    lines[1].recent_xfitted.append(right_fitx)
    lines[1].bestx = np.mean(lines[1].recent_xfitted)
    lines[1].recent_fitted.append(right_fit)
    lines[1].best_fit = np.mean(lines[1].recent_fitted)
    lines[1].current_fit = right_fit
    lines[1].radius_of_curvature = right_curverad
#     lines[1].line_base_pos = (midpoint - rightx[0])*xm_per_pix
    lines[1].line_base_pos = rightx[0]
    lines[1].allx = rightx
    lines[1].ally = righty
    if lines[1].detected:
        lines[1].diffs = right_fit - lines[1].recent_fitted[-1]
    if len(lines[1].recent_xfitted) > 3:
        lines[1].bestx = np.mean(lines[1].recent_xfitted[-3:], axis=0)
    
    img_center = (binary_warped.shape[1]) / 2
    dist_to_center = (img_center-(lines[0].line_base_pos+((lines[1].line_base_pos-lines[0].line_base_pos)/2.)))*xm_per_pix
    avg_curvature = (left_curverad+right_curverad)/2
#     print(lines[0].bestx)
    
    if len(lines[0].recent_xfitted) > 3 and len(lines[1].recent_xfitted) > 3:
#         return lines[0].bestx, lines[1].bestx, ploty, left_curverad, right_curverad, dist_to_center
        return lines[0].bestx, lines[1].bestx, ploty, avg_curvature, dist_to_center
    else:
#         return left_fitx, right_fitx, ploty, left_curverad, right_curverad, dist_to_center
        return left_fitx, right_fitx, ploty, avg_curvature, dist_to_center


In [23]:
class Line():
    def __init__(self):
        # was the line detected in the last iteration?
        self.detected = False  
        # x values of the last n fits of the line
        self.recent_xfitted = [] 
        #average x values of the fitted line over the last n iterations
        self.bestx = None     
        #list of recent polynomial coefficients over the last n iterations
        self.recent_fitted = []
        #polynomial coefficients averaged over the last n iterations
        self.best_fit = None  
        #polynomial coefficients for the most recent fit
        self.current_fit = [np.array([False])]  
        #radius of curvature of the line in some units
        self.radius_of_curvature = None 
        #distance in meters of vehicle center from the line
        self.line_base_pos = None 
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float') 
        #x values for detected line pixels
        self.allx = None  
        #y values for detected line pixels
        self.ally = None

In [24]:
objpts, imgpts = adjust_camera()
frame = mpimg.imread('test_images/test6.jpg')
data = FrameData(frame)
mtx, dist = calibrate(objpts, imgpts, data.img_size)

# Line detection
left_line = Line()
right_line = Line()
lines = (left_line, right_line)



midpoint = 0.

def process_image(frame):
    # Preprocess: framing, camera calibration
#     frame = mpimg.imread('test_images/test4.jpg')
    fr_data = FrameData(frame)
#     objpts, imgpts = adjust_camera(frame)
#     mtx, dist = calibrate(objpts, imgpts, fr_data.img_size)

    # Undistortion, Gradient/color screening, Perspective Transform
    dst = cv2.undistort(frame, mtx, dist, None, mtx)
    color, binary = screening(dst)
    prsp = Perspective(fr_data.img_size, fr_data.center_x, fr_data.center_y)
    binary_warped = transform(binary, fr_data.img_size, prsp)

    left_fitx, right_fitx, ploty, avg_curvature, dist_to_center = find_curves(binary_warped, lines, midpoint)

    # warped = np.copy(binary_warped)
    # Create an image to draw the lines on
    # warp_zero = np.zeros_like(warped).astype(np.uint8)
    warp_zero = np.zeros_like(binary_warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # left_fitx = left_line.recent_xfitted[-1]
    # right_fitx = right_line.recent_xfitted[-1]

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    Minv = cv2.getPerspectiveTransform(prsp.dst, prsp.src)

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, fr_data.img_size) 
    # Combine the result with the original image
    # result = cv2.addWeighted(undist, 1, newwarp, 0.3, 0)
    result = cv2.addWeighted(frame, 1, newwarp, 0.3, 0)
    
    font = cv2.FONT_HERSHEY_SIMPLEX
    if dist_to_center < 0:
        left_or_right = 'left'
    else:
        left_or_right = 'right'
    curverad_str = 'Radius of Curvature: '+str(int(avg_curvature))+'(m)'
    dist2cent_str = 'Vehicle is '+str(round(abs(dist_to_center),2))+'m '+left_or_right+' of center'
    cv2.putText(result,curverad_str,(10,110), font, 2,(255,255,255),2,cv2.LINE_AA)
    cv2.putText(result,dist2cent_str,(10,230), font, 2,(255,255,255),2,cv2.LINE_AA)
#     plt.imshow(result)
    return result

In [25]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML

challenge_output = 'challenge.mp4'
clip1 = VideoFileClip("challenge_video.mp4")
challenge_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time challenge_clip.write_videofile(challenge_output, audio=False)

[MoviePy] >>>> Building video challenge.mp4
[MoviePy] Writing video challenge.mp4


100%|██████████| 485/485 [02:41<00:00,  2.24it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: challenge.mp4 

CPU times: user 2min 28s, sys: 30.6 s, total: 2min 58s
Wall time: 2min 45s


In [26]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))

In [ ]:
# # Import everything needed to edit/save/watch video clips
# from moviepy.editor import VideoFileClip
# from IPython.display import HTML

# project_output = 'project.mp4'
# clip1 = VideoFileClip("project_video.mp4")
# project_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
# %time project_clip.write_videofile(project_output, audio=False)

In [ ]:
# HTML("""
# <video width="960" height="540" controls>
#   <source src="{0}">
# </video>
# """.format(project_output))